In [5]:
import pygame
import random

# Initialize pygame
pygame.init()

# Define colors
white = (255, 255, 255)
black = (0, 0, 0)
red = (213, 50, 80)
green = (0, 255, 0)
blue = (50, 153, 213)

# Set display dimensions
display_width = 600
display_height = 400

# Create game window
display = pygame.display.set_mode((display_width, display_height))
pygame.display.set_caption('Snake Game')

# Set game clock
clock = pygame.time.Clock()

# Snake block size and speed
snake_block = 10
snake_speed = 15

# Font styles
font_style = pygame.font.SysFont("bahnschrift", 25)
score_font = pygame.font.SysFont("comicsansms", 35)

class Snake:
    def __init__(self):
        self.reset()
        
    def reset(self):
        self.length = 1
        self.positions = [(display_width / 2, display_height / 2)]
        self.direction = random.choice(["UP", "DOWN", "LEFT", "RIGHT"])
        self.color = green
        self.score = 0
    
    def get_head_position(self):
        return self.positions[0]
    
    def move(self):
        head = self.get_head_position()
        x, y = head
        
        if self.direction == "UP":
            new_head = (x, y - snake_block)
        elif self.direction == "DOWN":
            new_head = (x, y + snake_block)
        elif self.direction == "LEFT":
            new_head = (x - snake_block, y)
        elif self.direction == "RIGHT":
            new_head = (x + snake_block, y)
            
        self.positions.insert(0, new_head)
        if len(self.positions) > self.length:
            self.positions.pop()
    
    def change_direction(self, new_direction):
        if (new_direction == "UP" and self.direction != "DOWN") or \
           (new_direction == "DOWN" and self.direction != "UP") or \
           (new_direction == "LEFT" and self.direction != "RIGHT") or \
           (new_direction == "RIGHT" and self.direction != "LEFT"):
            self.direction = new_direction
    
    def draw(self, surface):
        for pos in self.positions:
            pygame.draw.rect(surface, self.color, pygame.Rect(pos[0], pos[1], snake_block, snake_block))
    
    def check_collision(self):
        head = self.get_head_position()
        x, y = head
        
        # Check if snake hits the boundaries
        if x >= display_width or x < 0 or y >= display_height or y < 0:
            return True
        
        # Check if snake hits itself
        for pos in self.positions[1:]:
            if pos == head:
                return True
                
        return False
    
    def grow(self):
        self.length += 1
        self.score += 1

class Food:
    def __init__(self):
        self.position = (0, 0)
        self.color = red
        self.randomize_position()
        
    def randomize_position(self):
        self.position = (
            random.randint(0, (display_width - snake_block) // snake_block) * snake_block,
            random.randint(0, (display_height - snake_block) // snake_block) * snake_block
        )
    
    def draw(self, surface):
        pygame.draw.rect(surface, self.color, pygame.Rect(
            self.position[0], self.position[1], snake_block, snake_block))

def show_score(score):
    value = score_font.render("Score: " + str(score), True, black)
    display.blit(value, [10, 10])

def message(msg, color):
    mesg = font_style.render(msg, True, color)
    display.blit(mesg, [display_width / 6, display_height / 3])

def game_loop():
    snake = Snake()
    food = Food()
    
    running = True
    game_active = True
    
    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            
            if game_active:
                if event.type == pygame.KEYDOWN:
                    if event.key == pygame.K_UP:
                        snake.change_direction("UP")
                    elif event.key == pygame.K_DOWN:
                        snake.change_direction("DOWN")
                    elif event.key == pygame.K_LEFT:
                        snake.change_direction("LEFT")
                    elif event.key == pygame.K_RIGHT:
                        snake.change_direction("RIGHT")
            else:
                if event.type == pygame.KEYDOWN:
                    if event.key == pygame.K_q:
                        running = False
                    if event.key == pygame.K_c:
                        # Reset the game
                        snake.reset()
                        food.randomize_position()
                        game_active = True
        
        if game_active:
            snake.move()
            
            # Check if snake eats the food
            if snake.get_head_position() == food.position:
                food.randomize_position()
                snake.grow()
            
            # Check for collision
            if snake.check_collision():
                game_active = False
            
            # Draw everything
            display.fill(white)
            snake.draw(display)
            food.draw(display)
            show_score(snake.score)
        else:
            display.fill(white)
            message("Game Over! Press Q-Quit or C-Play Again", red)
            show_score(snake.score)
        
        pygame.display.update()
        clock.tick(snake_speed)
    
    pygame.quit()

# Start the game
if __name__ == "__main__":
    game_loop()